In [118]:
from datetime import datetime 
from itertools import chain
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, kendalltau

DATA_PATH = Path("../data")

# Task 1
Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl).Для унификации предлагаю:
- взять в тренировочный набор турниры с dateStart из 2019 года; 
- в тестовый — турниры с dateStart из 2020 года.


In [119]:
TRAIN_YEAR = 2019
TEST_YEAR = 2020

In [120]:
results = pd.read_pickle(DATA_PATH / "results.pkl")
players = pd.read_pickle(DATA_PATH / "players.pkl")
tournaments = pd.read_pickle(DATA_PATH / "tournaments.pkl")

In [121]:
tournaments_df = pd.DataFrame(tournaments).T
players_df = pd.DataFrame(players).T

In [122]:
def get_results(id_):
    t_results = results[id_]
    valid_t_results = [
        team
        for team in t_results
        if "mask" in team and team["mask"] and team["teamMembers"]
    ]
    if valid_t_results and (len(t_results) == len(valid_t_results)):
        return valid_t_results
    return None

tournaments_df["results"] = tournaments_df.id.apply(get_results)
tournaments_df = tournaments_df[~tournaments_df.results.isna()]
tournaments_df.sample(2)

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty,results
4211,4211,Критика способности суждения,2017-06-16T19:00:00+03:00,2017-06-20T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/50,"[{'id': 15875, 'name': 'Сергей', 'patronymic':...",{'dateRequestsAllowedTo': '2017-06-20T23:59:59...,"{'1': 13, '2': 13, '3': 13}","[{'team': {'id': 49804, 'name': 'Борский кораб..."
6137,6137,Синхронный кубок МГУ. Первая пара,2019-12-19T00:00:00+03:00,2019-12-25T23:50:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/53,"[{'id': 5990, 'name': 'Андрей', 'patronymic': ...",{'dateRequestsAllowedTo': '2019-12-24T12:00:00...,"{'1': 12, '2': 12, '3': 12}","[{'team': {'id': 27177, 'name': 'Призраки Коши..."


In [123]:
def get_players_data(tournaments_df):
    data = [
        (tm["player"]["id"], r["team"]["id"], tournament["id"], r["mask"], len(r["mask"]), r["position"])
        for _, tournament in tournaments_df.iterrows()
        for r in tournament.results
        for tm in r["teamMembers"]
    ]
    players_data = pd.DataFrame(
        data, 
        columns=["player_id", "team_id", "tournament_id", "mask", "questions_total", "position"]
    )
    return players_data

def get_questions(players_df):
    questions_nums = []
    for l in players_df['questions_total']:
        questions_nums.extend(range(1, l + 1))
        
    questions_df = pd.DataFrame({
        'player_id': np.repeat(players_df['player_id'], players_df['questions_total']),
        'tournament_id': np.repeat(players_df['tournament_id'], players_df['questions_total']),
        'team_id': np.repeat(players_df['team_id'], players_df['questions_total']),
        'position': np.repeat(players_df['position'], players_df['questions_total']),
        'question_num': questions_nums,
        'is_correct': list(chain.from_iterable(players_df['mask']))
    })

    questions_df = questions_df[~questions_df['is_correct'].isin(['?', 'X'])]
    questions_df.loc[:, 'is_correct'] = questions_df['is_correct'].astype(np.int8)
    return questions_df

In [124]:
tournaments_train = tournaments_df[pd.to_datetime(tournaments_df.dateStart, utc=True).dt.year == TRAIN_YEAR].reset_index(drop=True)
tournaments_test = tournaments_df[pd.to_datetime(tournaments_df.dateStart, utc=True).dt.year == TEST_YEAR].reset_index(drop=True)

players_train = get_players_data(tournaments_train)
players_test = get_players_data(tournaments_test)

# filter players (remove players who participate in only one set)
players_mask = (players_df.id.isin(players_train.player_id.unique())) & (players_df.id.isin(players_test.player_id.unique()))
players_df = players_df.loc[players_mask]
players_train = players_train.loc[players_train.player_id.isin(players_df.id)]
players_test = players_test.loc[players_test.player_id.isin(players_df.id)]

questions_train = get_questions(players_train)
questions_test = get_questions(players_test)

In [125]:
questions_train

,player_id,tournament_id,team_id,position,question_num,is_correct
0,6212,4772,45556,1.0,1,1
0,6212,4772,45556,1.0,2,1
0,6212,4772,45556,1.0,3,1
0,6212,4772,45556,1.0,4,1
0,6212,4772,45556,1.0,5,1
...,...,...,...,...,...,...
377396,104477,6191,50396,14.0,32,0
377396,104477,6191,50396,14.0,33,0
377396,104477,6191,50396,14.0,34,1
377396,104477,6191,50396,14.0,35,0


In [126]:
questions_train.to_csv(DATA_PATH / "train_questions_chgk.csv", index=False)
questions_test.to_csv(DATA_PATH / "test_questions_chgk.csv", index=False)

In [127]:
questions_train.player_id.nunique(), questions_test.player_id.nunique()

(19376, 19376)

# Task 2
Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. 
Замечания и подсказки:
- повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
- в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
- для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.


In [128]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

In [129]:
questions_train = pd.read_csv(DATA_PATH / "train_questions_chgk.csv")
questions_test = pd.read_csv(DATA_PATH / "test_questions_chgk.csv")

In [130]:
encoder = OneHotEncoder(handle_unknown="ignore")
player_ids = sorted(questions_train.player_id.unique())
train_data = encoder.fit_transform(questions_train[["player_id", "tournament_id", "question_num"]])
test_data = encoder.transform(questions_test[["player_id", "tournament_id", "question_num"]])

target_train = questions_train["is_correct"].values
target_test = questions_test["is_correct"].values

In [131]:
train_data.shape, test_data.shape

((12101414, 20526), (3640323, 20526))

In [132]:
baseline_model = LogisticRegression(solver="sag", n_jobs=-1)
baseline_model.fit(train_data, target_train)

LogisticRegression(n_jobs=-1, solver='sag')

In [133]:
def get_players_force(player_ids, lr_model):
    player_force = {}
    for p_id, coef in zip(player_ids, lr_model.coef_.flatten()):
        player_force[p_id] = coef
    return player_force

In [134]:
player_forces = get_players_force(player_ids, baseline_model)
players_df.loc[:, "player_force"] = players_df.loc[:, "id"].map(player_forces)
players_df = players_df.sort_values("player_force", ascending=False).reset_index(drop=True)
players_df.head(19380)

,id,name,patronymic,surname,player_force
0,27403,Максим,Михайлович,Руссо,2.585759
1,4270,Александра,Владимировна,Брутер,2.503009
2,28751,Иван,Николаевич,Семушин,2.466172
3,27822,Михаил,Владимирович,Савченков,2.399783
4,30152,Артём,Сергеевич,Сорожкин,2.347474
...,...,...,...,...,...
19371,218966,Михаил,Сергеевич,Мешков,-2.916364
19372,204714,Иван,Дмитриевич,Зарайский,-3.004244
19373,213896,Мехрубон,Одилжоювич,Тошматов,-3.037105
19374,207194,Ксения,Руслановна,Мерзлякова,-3.037113


# Task 3
Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. 
Поэтому:
- предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
- в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.


In [179]:
df = questions_test.copy()

# predict ranks
df["team_force"] = df.player_id.map(player_forces)
df = df.groupby(["tournament_id", "team_id"]).sum().reset_index()
df = df[["tournament_id", "team_id", "team_force"]]
df['pred_rank'] = df.groupby("tournament_id")['team_force'].rank(ascending=False)

# get true ranks
df = df.merge(
    questions_test[["tournament_id", "team_id", "position"]].drop_duplicates(), on=["tournament_id", "team_id"]
).rename({"position": "true_rank"}, axis=1)

# Count Correlations
spearman_corrs = []
kendall_corrs = []
for tournament_id, g in df.groupby("tournament_id"):
    if len(g) == 1:  # tournament_id 6362
        continue
    spearman_corrs.append(spearmanr(g.true_rank.values, g.pred_rank.values).correlation)
    kendall_corrs.append(kendalltau(g.true_rank.values, g.pred_rank.values).correlation)

spearman = np.mean(spearman_corrs)
kendrall = np.mean(kendall_corrs)

print(f"Spearman: {spearman:.4f}")
print(f"Kendall: {kendrall:.4f}")

Spearman: 0.7893
Kendall: 0.6303
